In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [2]:
#Load Dataset
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
## PreProcess the data
### Drop irrelevant columns
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1) #axis=1 column wise
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
data['Gender'].value_counts()

Gender
Male      5457
Female    4543
Name: count, dtype: int64

In [ ]:
## Encode categorical variables
label_encoder_gender = LabelEncoder() ## Here We are training the encoder based on categorical value
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [6]:
data['Gender'].value_counts()

Gender
1    5457
0    4543
Name: count, dtype: int64

In [7]:
## One hot encode 'Geography' column
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [8]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]], shape=(10000, 3))

In [9]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [10]:
geo_encoder_df = pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [11]:
geo_encoder_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [12]:
##Combine one hot encoded columns with the original data

data = pd.concat([data.drop('Geography',axis=1),geo_encoder_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [ ]:
# save the encode

with open('label_encode_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

In [14]:
print(label_encoder_gender.classes_)

['Female' 'Male']


In [15]:
## Divide the dataset into dependent and independent features
x =  data.drop('Exited',axis=1)
y = data['Exited']

# Split the data into training and testing sets
x_train,x_test,y_train,y_test = train_test_split( x,y,test_size=0.2, random_state=32)

# Scale these features
scalar = StandardScaler()
x_train = scalar.fit_transform(x_train)
x_test = scalar.fit_transform(x_test)


In [16]:
x_train

array([[-1.24955805, -1.10191942,  0.19630082, ..., -0.996257  ,
        -0.58062195,  1.73320628],
       [-0.33729879, -1.10191942,  0.1011362 , ...,  1.00375706,
        -0.58062195, -0.57696537],
       [ 0.64752654, -1.10191942,  1.90926396, ...,  1.00375706,
        -0.58062195, -0.57696537],
       ...,
       [-0.49279753, -1.10191942,  0.29146544, ..., -0.996257  ,
        -0.58062195,  1.73320628],
       [-0.08850081,  0.90750738, -1.99248542, ..., -0.996257  ,
         1.72229108, -0.57696537],
       [-1.76788717,  0.90750738,  0.00597158, ..., -0.996257  ,
        -0.58062195,  1.73320628]], shape=(8000, 12))

In [17]:
x_test

array([[-0.00508813,  0.93228691, -1.81288002, ..., -1.02943297,
        -0.57119169,  1.78195548],
       [ 0.47753935, -1.07263117, -0.08266579, ..., -1.02943297,
         1.75072575, -0.56118125],
       [ 0.8369428 ,  0.93228691,  0.30182626, ..., -1.02943297,
         1.75072575, -0.56118125],
       ...,
       [ 0.82667413,  0.93228691, -0.1787888 , ..., -1.02943297,
         1.75072575, -0.56118125],
       [ 0.24135995,  0.93228691,  0.68631831, ...,  0.97140856,
        -0.57119169, -0.56118125],
       [-0.11804349,  0.93228691, -1.14001893, ..., -1.02943297,
         1.75072575, -0.56118125]], shape=(2000, 12))

In [ ]:
#save the scalar

with open('scalar.pkl','wb') as file:
    pickle.dump(scalar,file)

## **ANN IMPLEMENTATION**

In [19]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

d:\Rajat\Installer\Anaconda\envs\myenv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [20]:
## Build our ANN model

#step1 : initialise the sequential model

model = Sequential([
    Dense(64,activation='relu',input_shape = (x_train.shape[1],)),  #HL1 with 64 neuron connected with input layer 
    Dense(32,activation='relu'), #HL2
    Dense(1,activation='sigmoid')  ## Output layer
])

d:\Rajat\Installer\Anaconda\envs\myenv\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
# one way
import tensorflow
optim = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()
loss

<LossFunctionWrapper(<function binary_crossentropy at 0x0000025DCAA9AE80>, kwargs={'from_logits': False, 'label_smoothing': 0.0, 'axis': -1})>

In [23]:
# Second way
model.compile(optimizer=optim,loss=loss,metrics=['accuracy'])

In [24]:
## Set up the tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard 
log_dir = "log/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [25]:
## Setup Early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)


In [26]:
## Train the Model
history = model.fit(
    x_train,y_train,validation_data=(x_test,y_test),epochs=100,
    callbacks = [tensorflow_callback,early_stopping_callback    ]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8338 - loss: 0.3984 - val_accuracy: 0.8325 - val_loss: 0.3937
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8593 - loss: 0.3492 - val_accuracy: 0.8390 - val_loss: 0.3786
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8646 - loss: 0.3414 - val_accuracy: 0.8400 - val_loss: 0.3740
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8651 - loss: 0.3388 - val_accuracy: 0.8485 - val_loss: 0.3641
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8646 - loss: 0.3364 - val_accuracy: 0.8455 - val_loss: 0.3620
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8652 - loss: 0.3342 - val_accuracy: 0.8435 - val_loss: 0.3630
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8644 - loss: 0.3299 - val_accuracy: 0.8515 - val_loss: 0.3585
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8690 - loss: 0.3281 - val_accu

In [27]:
model.save('model.h5')

In [28]:
## Load tensorboard extension
%load_ext tensorboard

In [29]:
%tensorboard --logdir log/fit20260105-160130 --port 6011

Reusing TensorBoard on port 6011 (pid 4940), started 0:39:09 ago. (Use '!kill 4940' to kill it.)

In [30]:
## Load the pickle file